# This code demonstrates that the layerwise computation is accurate in the same way that the easy ntk experiment is. We also verify that the easy_NTK gives the same answer as layerwise_NTK, lending credibility to both.

In [1]:
import numpy as np
import torch
import random

import matplotlib.pyplot as plt


import torch
from torch import nn, optim
from torch import load
from torch.nn import functional as F
from torch import autograd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#from ..easy_ntk import calculate_NTK
#from einops import rearrange

import time

import sys
from pathlib import Path

from numba import njit
from numba.typed import List

%matplotlib inline
%load_ext line_profiler
%load_ext memory_profiler

In [2]:
import copy

def _del_nested_attr(obj, names):
    """
    Deletes the attribute specified by the given list of names.
    For example, to delete the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'])
    """
    if len(names) == 1:
        delattr(obj, names[0])
    else:
        _del_nested_attr(getattr(obj, names[0]), names[1:])

def _set_nested_attr(obj, names, value):
    """
    Set the attribute specified by the given list of names to value.
    For example, to set the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'], value)
    """
    if len(names) == 1:
        setattr(obj, names[0], value)
    else:
        _set_nested_attr(getattr(obj, names[0]), names[1:], value)

def extract_weights(mod):
    """
    This function removes all the Parameters from the model and
    return them as a tuple as well as their original attribute names.
    The weights must be re-loaded with `load_weights` before the model
    can be used again.
    Note that this function modifies the model in place and after this
    call, mod.parameters() will be empty.
    """
    orig_params = tuple(mod.parameters())
    # Remove all the parameters in the model
    names = []
    for name, p in list(mod.named_parameters()):
        _del_nested_attr(mod, name.split("."))
        names.append(name)

    # Make params regular Tensors instead of nn.Parameter
    params = tuple(p.detach().requires_grad_() for p in orig_params)
    return params, names

def load_weights(mod, names, params):
    """
    Reload a set of weights so that `mod` can be used again to perform a forward pass.
    Note that the `params` are regular Tensors (that can have history) and so are left
    as Tensors. This means that mod.parameters() will still be empty after this call.
    """
    for name, p in zip(names, params):
        _set_nested_attr(mod, name.split("."), p)
        
def calculate_NTK(model,x,device='cpu',MODE='samples'):
    """
    INPUTS:
        model: torch.nn.Module 
        x: torch.Tensor
        device: 'cpu',
        MODE: 'minima'
    
    OUTPUTS:
        NTK: torch.Tensor
    
    Calculates the NTK for a model, p_dict a state dictionary, and x, a single tensor fed into the model
    
    The NTK is the grammian of the Jacobian of the model output to w.r.t. the weights of the model
    
    This function will output the NTK such that the minima matrix size is used. If the Jacobian is an NxM
    matrix, then the NTK is formulated so that if N < M; NTK is NxN. If M<N, then NTK is MxM.
    
    #EXAMPLE USAGE:
    device='cpu'
    model = MODEL() #a torch.nn.Module object 
    model.to(device)
    state_dict = model.state_dict()

    x_test = np.ones((100,1,28,28),dtype=np.float32)
    x_test = torch.from_numpy(x_test)

    NTK = calculate_NTK(model,x_test)
    """
    if not(MODE in ['minima','samples','params']):
        raise ValueError("MODE must be one of 'minima','samples','params'")
    
    x = x.to(device)
    x.requires_grad=False
    N = x.shape[0]
    M = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    #We need to create a clone of the model or else we make it unusable as part of the trickery 
    #to get pytorch to do what we want. Unforutantely, this exlcludes super big models. but, eh.
    model_clone = copy.deepcopy(model)
    
    params, names = extract_weights(model_clone)
    def model_ntk(*args,model=model_clone, names=names):
        params = tuple(args)
        load_weights(model, names, params)
        return model(x)
    
    Js = torch.autograd.functional.jacobian(model_ntk, tuple(params), create_graph=False, vectorize=True)
    
    Js = list(Js)
    #Js = [element for tupl in Js for element in tupl]
    #collapse the tensors
    for i,tensor in enumerate(Js):
        Js[i] = tensor.reshape(N,-1)
    
    J = torch.cat(Js,axis=1)
    
    if MODE=='minima':
        if N < M: #if datasize points is less than number of parameters:
            NTK = torch.matmul(J,J.T)

        if N >= M:#if number of parameters is less than datasize:
            NTK = torch.matmul(J.T,J)
    elif MODE=='samples':
        NTK = torch.matmul(J,J.T)
    elif MODE=='params':
        NTK = torch.matmul(J.T,J)
    
    return NTK

In [3]:
def relu(X,normalize=False):
    X = F.relu(X)
    if normalize:
        return np.sqrt(2*np.pi/(np.pi-1))*(X-1/np.sqrt(2*np.pi))
    else:
        return X
    

# #Identity
def activation(x):
    return x

# @njit
def d_activation(x):
    return np.ones(np.shape(x),dtype=np.float32) 


#Tanh
#def activation(x):
#    return torch.tanh(x)

#@njit
#def d_activation(x):
#    return np.cosh(x)**-2

In [4]:
SEED = 1

In [15]:
@njit #no parallel transformation available ;#fasterer
def cross(X):
    return X.T.dot(X)

def compute_NTK(Ws, Xs, d_int, d_array):#L counts from 1 to number of layers.
    '''
    I should add some docstring
    
    Ws, a list of the weights as np.array type np.float32,                          [W1, W2, W3 ... W]
    Xs, a list of the conjugate kernels as np.array type np.float32,            [X0, X1, X2, ... XL]
    d_int, a list of the dimensionality of X_l as int64,                        [d0, d1, d2, ... dL]
    d_array, a list of the dimensionality of X_l, as np.array type np.float 32, [d0, d1, d2, ... dL] 
    all of this is neccessary because numba doesnt like type conversion.
    
    outputs the NTK as a np.array of type np.float32
    '''
    L = len(Xs)-1 #number of layers, Xs goes from inputs to right before outputs; X_0 is the input, X_L CK
    
    n = Xs[0].shape[1] #number of datapoints
    
    #holds the derivatives, first value is empty list...?; just a spacer, replace with array
    Ds = [np.array([[0.0]],dtype=np.float32)] 
    for l in range(L):
        Ds.append(d_activation(np.dot(Ws[l],Xs[l])))
    
    #The first term is just conjugate kernel
    KNTK = cross(Xs[L])
    #print('3: ',KNTK)
    for l in range(1,L+1):#l counts layers going forward from 1...
        #we are going to construct terms that look like ( S^T S ) * (X^T X)
        XtX = cross(Xs[l-1])
        S = np.zeros((d_int[l],n),dtype=np.float32)
        for i in range(n):
            
            #this is always the same, could be calculated once and saved
            s = Ws[-1].T.reshape(-1)/np.sqrt(d_array[L]) #has shape input to last layer.

            for k in range(L,l-1,-1): #counts backwards from L to l, inclusive both
                s = Ds[k][:,i]*s
                if k > l:
                    s = np.dot(s,Ws[k-1])/np.sqrt(d_array[k-1])
            S[:,i] = s
        #print(l, cross(S) * XtX)
        print(l, cross(S) * XtX)
        KNTK += cross(S) * XtX
    return KNTK

def compute_NTK_new(Ws, Xs, d_int, d_array):
    L = len(Xs)-1 #number of layers, Xs goes from inputs to right before outputs; X_0 is the input, X_L CK

    n = Xs[0].shape[1] #number of datapoints

    #holds the derivatives, first value is empty list...?; just a spacer, replace with array
    Ds = [np.array([[0.0]],dtype=np.float32)] 
    for l in range(L):
        Ds.append(d_activation(np.dot(Ws[l],Xs[l])))
    print('length Ds: ',len(Ds))
    #The first term is just conjugate kernel
    KNTK = cross(Xs[L])

    ####################
    for l in range(1,L+1):#l counts layers going forward from 1...
        #we are going to construct terms that look like ( S^T S ) * (X^T X)
        XtX = cross(Xs[l-1])
        S = np.expand_dims(Ws[-1].T.reshape(-1)/np.sqrt(d_array[L]),axis=1) #has shape input to last layer.
        print('l : ',l)
        for k in range(L,l-1,-1): #counts backwards from l
            print('k : ',k)
            S = Ds[k]*S
            if k > l:
                S = np.dot(S.T,Ws[k-1]).T/np.sqrt(d_array[k-1])
        KNTK += cross(S) * XtX
    return KNTK


# L, should be 3

# l :  1
# k :  3
# k :  2
# k :  1

# l :  2
# k :  3
# k :  2

# l :  3
# k :  3

In [16]:
def NTK_weights(m):
    if isinstance(m, nn.Linear):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]
    if isinstance(m, nn.Conv2d):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]

In [17]:
#Layerwise Needs each conjugate Kernel
class dumb_small_layerwise(torch.nn.Module):
    '''
    simple network for test cases
    '''
    def __init__(self,):
        super(dumb_small_layerwise, self).__init__()
        
        self.d1 = torch.nn.Linear(5,256,bias=False)
        self.d2 = torch.nn.Linear(256,256,bias=False)
        self.d3 = torch.nn.Linear(256,256,bias=False)
        self.d4 = torch.nn.Linear(256,1,bias=False)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0)) / np.sqrt(256)
        x_2 = activation(self.d2(x_1)) / np.sqrt(256)
        x_3 = activation(self.d3(x_2)) / np.sqrt(256)
        x_4 = self.d4(x_3)
        
        return x_4, x_3, x_2, x_1, x_0
    
# Easy NTK expects one output alone
class dumb_small(torch.nn.Module):
    '''
    simple network for test cases
    '''
    def __init__(self,):
        super(dumb_small, self).__init__()
        
        self.d1 = torch.nn.Linear(5,256,bias=False)
        self.d2 = torch.nn.Linear(256,256,bias=False)
        self.d3 = torch.nn.Linear(256,256,bias=False)
        self.d4 = torch.nn.Linear(256,1,bias=False)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0)) / np.sqrt(256)
        x_2 = activation(self.d2(x_1)) / np.sqrt(256)
        x_3 = activation(self.d3(x_2)) / np.sqrt(256)
        x_4 = self.d4(x_3)
        return x_4

In [18]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cpu'

model_small = dumb_small()
model_small.to(device)
model_small.apply(NTK_weights)

#Reset the seed and 
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)


model_layerwise = dumb_small_layerwise()
model_layerwise.to(device)
model_layerwise.apply(NTK_weights)

x_test = np.random.normal(0,1,(10,5)).astype(np.float32)
x_test = torch.from_numpy(x_test)

torch.Size([256, 5])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([1, 256])
torch.Size([256, 5])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([1, 256])


# Now compute 3 ways:

#### by hand

In [19]:
# W1 = model_small.d1.weight.detach().numpy().T
# W2 = model_small.d2.weight.detach().numpy().T

# print(W1.shape)

# A00 = W1[0,0] 
# A01 = W1[0,1]
# A10 = W1[1,0] 
# A11 = W1[1,1]
# A20 = W1[2,0]
# A21 = W1[2,1]

# B00 = W2[0,0]
# B10 = W2[1,0]

# X00 = x_test[0,0]
# X01 = x_test[0,1]
# X02 = x_test[0,2]
# X10 = x_test[1,0]
# X11 = x_test[1,1]
# X12 = x_test[1,2]

# J = np.array([[X00*B00, X01*B00, X02*B00, X00*B10, X01*B10, X02*B10, X00*A00 + X01*A10 + X02*A20, X00*A01 + X01*A11 + X02*A21],
#               [X10*B00, X11*B00, X12*B00, X10*B10, X11*B10, X12*B10, X10*A00 + X11*A10 + X12*A20, X10*A01 + X11*A11 + X12*A21]])

# J = J / np.sqrt(2) #in this 2 layer linear network, this is okay.

# NTK_byhand = (J @ J.T) 

#### Layerwise

In [20]:
x_4, x_3, x_2, x_1, x_0 = model_layerwise(x_test)

Ws = []
Ws.append(model_layerwise.d1.weight.detach().numpy().astype(np.float32))
Ws.append(model_layerwise.d2.weight.detach().numpy().astype(np.float32))
Ws.append(model_layerwise.d3.weight.detach().numpy().astype(np.float32))
Ws.append(model_layerwise.d4.weight.detach().numpy().astype(np.float32))

Xs = [] # Xs are shape (output x #DP) ; however, typical python notation is reversed, so we take transpose here
Xs.append(x_0.detach().numpy().T.astype(np.float32))
Xs.append(x_1.detach().numpy().T.astype(np.float32))
Xs.append(x_2.detach().numpy().T.astype(np.float32))
Xs.append(x_3.detach().numpy().T.astype(np.float32))

ds_int = []
ds_int.append(0)
ds_int.append(256)
ds_int.append(256)
ds_int.append(256)

ds_array = []
ds_array.append(np.array([0.0],dtype=np.float32)) #first element is the input length
ds_array.append(np.array([256.0],dtype=np.float32))
ds_array.append(np.array([256.0],dtype=np.float32)) #the remaining elements are the output lengths, but omit the last output length assumed 1.
ds_array.append(np.array([256.0],dtype=np.float32))

In [21]:
NTK_layerwise = compute_NTK(List(Ws), List(Xs), List(ds_int), List(ds_array))

1 [[ 5.6825566  -5.430897    5.690123   -0.83005065 -2.9812381   0.22461934
  -0.20998777 -0.6562718  -1.2542443  -2.0309448 ]
 [-5.430897    9.94341    -7.792306    3.028438    4.137024    2.0289896
   1.4452263  -1.9200761  -0.01225542  0.09289722]
 [ 5.690123   -7.792306    8.667182   -0.35607603 -3.753958    0.28389212
  -0.4474241   2.6394753   1.309752   -1.47562   ]
 [-0.83005065  3.028438   -0.35607603  2.573817    0.84052926  2.0713265
   1.1005363   0.55091065  1.2269455  -0.8878967 ]
 [-2.9812381   4.137024   -3.753958    0.84052926  4.8143535   0.1218503
  -1.468686    0.4912028  -0.6373993   3.7537062 ]
 [ 0.22461934  2.0289896   0.28389212  2.0713265   0.1218503   1.8730148
   1.1329446  -0.13625196  0.56144756 -1.3770355 ]
 [-0.20998777  1.4452263  -0.4474241   1.1005363  -1.468686    1.1329446
   2.4877617  -1.762147   -0.51036155 -1.7449169 ]
 [-0.6562718  -1.9200761   2.6394753   0.55091065  0.4912028  -0.13625196
  -1.762147    5.044641    4.0123224   1.0337812 ]
 [-

In [22]:
NTK_layerwise_new = compute_NTK_new(List(Ws), List(Xs), List(ds_int), List(ds_array))

length Ds:  4
l :  1
k :  3
k :  2
k :  1
l :  2
k :  3
k :  2
l :  3
k :  3


In [23]:
NTK_layerwise

array([[ 21.9125   , -19.73215  ,  21.335274 ,  -2.494825 , -11.171752 ,
          1.6160681,   0.7629754,  -4.5374703,  -6.928875 ,  -7.104703 ],
       [-19.73215  ,  36.60446  , -28.594894 ,  11.1278   ,  12.858362 ,
          7.630098 ,   5.757512 ,  -6.8720016,   2.0185657,  -3.425946 ],
       [ 21.335274 , -28.594894 ,  32.979218 ,  -0.2559224, -12.720112 ,
          1.9158003,  -1.0168245,   9.647783 ,   3.6374817,  -3.4760442],
       [ -2.494825 ,  11.1278   ,  -0.2559224,  10.300727 ,   1.8668735,
          8.448566 ,   4.7998934,   2.3872528,   5.6345854,  -5.111598 ],
       [-11.171752 ,  12.858362 , -12.720112 ,   1.8668735,  16.888258 ,
         -0.8336451,  -6.650405 ,   3.9538937,  -0.4944474,  13.900022 ],
       [  1.6160681,   7.630098 ,   1.9158003,   8.448566 ,  -0.8336451,
          7.82065  ,   5.2693157,  -0.9256127,   2.445969 ,  -6.921986 ],
       [  0.7629754,   5.757512 ,  -1.0168245,   4.7998934,  -6.650405 ,
          5.2693157,  10.380568 ,  -8.131702 

In [24]:
#%lprun -f compute_NTK_new compute_NTK_new(List(Ws), List(Xs), List(ds_int), List(ds_array))

In [25]:
#%%timeit 
#compute_NTK_new(List(Ws), List(Xs), List(ds_int), List(ds_array))

In [26]:
#%%timeit
#compute_NTK(List(Ws), List(Xs), List(ds_int), List(ds_array))

In [27]:
#%memit compute_NTK(List(Ws), List(Xs), List(ds_int), List(ds_array))

In [28]:
#%memit compute_NTK_new(List(Ws), List(Xs), List(ds_int), List(ds_array))

#### Jacobian autograd

In [29]:
NTK_easy = calculate_NTK(model_small,x_test).detach().numpy()

# Compare

In [30]:
#print(NTK_byhand)

In [31]:
print(NTK_easy)

[[ 21.912504   -19.732151    21.33528     -2.4948232  -11.171747
    1.6160682    0.76297545  -4.537472    -6.9288745   -7.104703  ]
 [-19.732151    36.604454   -28.594902    11.127797    12.858372
    7.6300993    5.7575135   -6.872        2.0185654   -3.4259481 ]
 [ 21.33528    -28.594902    32.979202    -0.25592268 -12.72011
    1.9158006   -1.016825     9.647791     3.6374834   -3.476046  ]
 [ -2.4948232   11.127797    -0.25592268  10.300735     1.8668748
    8.44857      4.7998943    2.3872535    5.634585    -5.1116004 ]
 [-11.171747    12.858372   -12.72011      1.8668748   16.888258
   -0.833645    -6.650404     3.9538946   -0.49444753  13.900022  ]
 [  1.6160682    7.6300993    1.9158006    8.44857     -0.833645
    7.8206496    5.269317    -0.9256128    2.4459682   -6.9219923 ]
 [  0.76297545   5.7575135   -1.016825     4.7998943   -6.650404
    5.269317    10.380568    -8.1317005   -2.6072366   -8.616312  ]
 [ -4.537472    -6.872        9.647791     2.3872535    3.9538946
   

In [32]:
print(NTK_layerwise)

[[ 21.9125    -19.73215    21.335274   -2.494825  -11.171752    1.6160681
    0.7629754  -4.5374703  -6.928875   -7.104703 ]
 [-19.73215    36.60446   -28.594894   11.1278     12.858362    7.630098
    5.757512   -6.8720016   2.0185657  -3.425946 ]
 [ 21.335274  -28.594894   32.979218   -0.2559224 -12.720112    1.9158003
   -1.0168245   9.647783    3.6374817  -3.4760442]
 [ -2.494825   11.1278     -0.2559224  10.300727    1.8668735   8.448566
    4.7998934   2.3872528   5.6345854  -5.111598 ]
 [-11.171752   12.858362  -12.720112    1.8668735  16.888258   -0.8336451
   -6.650405    3.9538937  -0.4944474  13.900022 ]
 [  1.6160681   7.630098    1.9158003   8.448566   -0.8336451   7.82065
    5.2693157  -0.9256127   2.445969   -6.921986 ]
 [  0.7629754   5.757512   -1.0168245   4.7998934  -6.650405    5.2693157
   10.380568   -8.131702   -2.6072369  -8.616311 ]
 [ -4.5374703  -6.8720016   9.647783    2.3872528   3.9538937  -0.9256127
   -8.131702   22.106916   17.110071    6.397989 ]
 [ -

In [33]:
NTK_layerwise_new

array([[ 21.912498  , -19.73215   ,  21.335274  ,  -2.494825  ,
        -11.171751  ,   1.6160681 ,   0.7629754 ,  -4.53747   ,
         -6.928875  ,  -7.104703  ],
       [-19.73215   ,  36.60446   , -28.594893  ,  11.1278    ,
         12.858361  ,   7.6300974 ,   5.757511  ,  -6.8720016 ,
          2.0185657 ,  -3.425946  ],
       [ 21.335274  , -28.594893  ,  32.979218  ,  -0.25592238,
        -12.72011   ,   1.9158003 ,  -1.0168244 ,   9.647781  ,
          3.6374817 ,  -3.4760442 ],
       [ -2.494825  ,  11.1278    ,  -0.25592238,  10.300726  ,
          1.8668733 ,   8.448566  ,   4.7998934 ,   2.3872526 ,
          5.6345854 ,  -5.111598  ],
       [-11.171751  ,  12.858361  , -12.72011   ,   1.8668733 ,
         16.888256  ,  -0.8336451 ,  -6.650404  ,   3.9538934 ,
         -0.4944474 ,  13.900022  ],
       [  1.6160681 ,   7.6300974 ,   1.9158003 ,   8.448566  ,
         -0.8336451 ,   7.8206496 ,   5.2693157 ,  -0.9256127 ,
          2.445969  ,  -6.921986  ],
       [  

In [34]:
#!!! success!
np.allclose(NTK_layerwise_new,NTK_easy)

True

# Can we get pytorch to give us the matrices with the correct kinds of derivative?

# verify hypothesis by increasing number of terms...

### also lets us see how this fares on more than one datapoint... it'll be alot of calls to backward...

### https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html

### note to future self, since these are partials pretty sure we can use inputs = w1 and that will make it so the gradients are computed alittle more efficiently

### and if inputs does work like that, then the reallly best thing would be to find a way to get vector gradients working.


### also note pytorch doesn't support autograd on non-scalar values, which means we are forced to call on backwards multiple times

In [20]:
model_layerwise.zero_grad()
x_4, x_3, x_2, x_1, x_0 = model_layerwise(x_test)

In [21]:
model_layerwise.zero_grad()
x_4 = model_small(x_test)

In [23]:
#this method agrees between model layerwise and model small; meaning that the calculation is indepdent of those
#two models. the insinuation is somehting is wrong with both my methods for calculating,--- the same thing, since
#they agree with one another.

#in the future we would iterate over layers instead of like this...
layer_components_w1 = [] #and we can make these arrays since we will know their size and slice into them
# and we would only do this one array element at a time.
layer_components_w2 = []
layer_components_w3 = []
layer_components_w4 = []
for output in x_4:
#    model_layerwise.zero_grad()
    model_small.zero_grad()
    
    output.backward(retain_graph=True)

#     w3_grad = model_layerwise.d3.weight.grad.detach().numpy()
#     w2_grad = model_layerwise.d2.weight.grad.detach().numpy()
#     w1_grad = model_layerwise.d1.weight.grad.detach().numpy()
    w4_grad = model_small.d4.weight.grad.detach().numpy()
    w3_grad = model_small.d3.weight.grad.detach().numpy()
    w2_grad = model_small.d2.weight.grad.detach().numpy()
    w1_grad = model_small.d1.weight.grad.detach().numpy()


    layer_components_w1.append(w1_grad.reshape(-1).copy())
    layer_components_w2.append(w2_grad.reshape(-1).copy())
    layer_components_w3.append(w3_grad.reshape(-1).copy())
    layer_components_w4.append(w4_grad.reshape(-1).copy())

In [24]:
layer_components_w1 = np.array(layer_components_w1)
layer_components_w2 = np.array(layer_components_w2)
layer_components_w3 = np.array(layer_components_w3)
layer_components_w4 = np.array(layer_components_w4)

In [25]:
autograd_NTK = layer_components_w1 @ layer_components_w1.T+\
    layer_components_w2 @ layer_components_w2.T+\
    layer_components_w3 @ layer_components_w3.T+\
    layer_components_w4 @ layer_components_w4.T

In [29]:
layer_components_w2 @ layer_components_w2.T

array([[ 0.26212782, -0.13915719,  0.17513433, -0.01963228, -0.10402071,
         0.02361404,  0.00735725, -0.01632607, -0.04401862, -0.04432306],
       [-0.13915719,  0.26258352, -0.1740298 ,  0.08608555,  0.09840427,
         0.05940237,  0.03724238, -0.04245349,  0.005094  , -0.00354181],
       [ 0.17513433, -0.1740298 ,  0.25934973,  0.00085505, -0.08736758,
         0.02161472, -0.00594713,  0.05563299,  0.01757241, -0.01121507],
       [-0.01963228,  0.08608555,  0.00085505,  0.2057347 ,  0.03086133,
         0.17467298,  0.06747642,  0.02441898,  0.06299526, -0.04137547],
       [-0.10402071,  0.09840427, -0.08736758,  0.03086133,  0.27646452,
        -0.00478765, -0.08475629,  0.03604071, -0.00832839,  0.12688056],
       [ 0.02361404,  0.05940237,  0.02161472,  0.17467298, -0.00478765,
         0.1864023 ,  0.08384367, -0.00451578,  0.03314169, -0.06775427],
       [ 0.00735725,  0.03724238, -0.00594713,  0.06747642, -0.08475629,
         0.08384367,  0.18186687, -0.08083846

In [ ]:
# 2 [[ 0.23109226 -0.13013916  0.17527533 -0.01816671 -0.09664266  0.02248176
#    0.00669326 -0.01512574 -0.04260881 -0.04601773]
#  [-0.13013916  0.24945484 -0.17660742  0.08622429  0.10080978  0.06020794
#    0.03772704 -0.04323682  0.00542523 -0.00371261]
#  [ 0.17527533 -0.17660742  0.27889174  0.00089781 -0.09548703  0.02274703
#   -0.00634935  0.0582776   0.01933318 -0.01291431]
#  [-0.01816671  0.08622429  0.00089781  0.22381395  0.03286307  0.18501176
#    0.06594371  0.02559525  0.07113553 -0.0444255 ]
#  [-0.09664266  0.10080978 -0.09548703  0.03286307  0.28040892 -0.00513606
#   -0.08478455  0.03666005 -0.00888365  0.14024408]
#  [ 0.02248176  0.06020794  0.02274703  0.18501176 -0.00513606  0.19367565
#    0.08173902 -0.00470114  0.03726654 -0.07162631]
#  [ 0.00669326  0.03772704 -0.00634935  0.06594371 -0.08478455  0.08173902
#    0.17753676 -0.08210288 -0.02242952 -0.0883496 ]
#  [-0.01512574 -0.04323682  0.0582776   0.02559525  0.03666005 -0.00470114
#   -0.08210288  0.22881114  0.16830118  0.04913284]
#  [-0.04260881  0.00542523  0.01933318  0.07113553 -0.00888365  0.03726654
#   -0.02242952  0.16830118  0.25831077 -0.02846634]
#  [-0.04601773 -0.00371261 -0.01291431 -0.0444255   0.14024408 -0.07162631
#   -0.0883496   0.04913284 -0.02846634  0.23442203]]

# 1 [[ 4.3372995e-01 -1.2680553e-01  2.4192266e-01 -3.2461844e-02
#   -9.3728907e-02  1.0109340e-02 -7.7008591e-03 -1.9981360e-02
#   -5.2510254e-02 -6.3229941e-02]
#  [-1.2680553e-01  4.2842412e-01 -2.2196750e-01  9.7295851e-02
#    9.3750760e-02  6.9861203e-02  3.1264532e-02 -2.7425971e-02
#   -2.4413578e-04  1.6981254e-03]
#  [ 2.4192266e-01 -2.2196750e-01  5.7009739e-01 -1.2536406e-02
#   -1.0858573e-01  1.0893183e-02 -1.3385277e-02  5.4588843e-02
#    3.9020326e-02 -4.5721728e-02]
#  [-3.2461844e-02  9.7295851e-02 -1.2536406e-02  3.0397519e-01
#    3.8576838e-02  2.1457963e-01  6.2807687e-02  2.4017636e-02
#    6.5343060e-02 -2.7964469e-02]
#  [-9.3728907e-02  9.3750760e-02 -1.0858573e-01  3.8576838e-02
#    3.9145073e-01  6.3925628e-03 -5.8467742e-02  1.3922420e-02
#   -2.0138728e-02  1.5643328e-01]
#  [ 1.0109340e-02  6.9861203e-02  1.0893183e-02  2.1457963e-01
#    6.3925628e-03  2.1332595e-01  7.2265819e-02 -5.4552890e-03
#    2.7105577e-02 -5.9520919e-02]
#  [-7.7008591e-03  3.1264532e-02 -1.3385277e-02  6.2807687e-02
#   -5.8467742e-02  7.2265819e-02  2.3293228e-01 -5.9420872e-02
#   -1.6525010e-02 -7.4892543e-02]
#  [-1.9981360e-02 -2.7425971e-02  5.4588843e-02  2.4017636e-02
#    1.3922420e-02 -5.4552890e-03 -5.9420872e-02  3.5255811e-01
#    2.0547493e-01  2.8280120e-02]
#  [-5.2510254e-02 -2.4413578e-04  3.9020326e-02  6.5343060e-02
#   -2.0138728e-02  2.7105577e-02 -1.6525010e-02  2.0547493e-01
#    3.9798790e-01 -4.7951743e-02]
#  [-6.3229941e-02  1.6981254e-03 -4.5721728e-02 -2.7964469e-02
#    1.5643328e-01 -5.9520919e-02 -7.4892543e-02  2.8280120e-02
#   -4.7951743e-02  3.7898111e-01]]

# 3 [[ 0.2271919  -0.11885324  0.15557915  0.00355438 -0.09601458  0.04310704
#    0.01562474 -0.02591271 -0.04562492 -0.06052731]
#  [-0.11885324  0.21490423 -0.15661456  0.07115156  0.07242595  0.05466134
#    0.04481084 -0.0430408   0.00233283 -0.02437924]
#  [ 0.15557915 -0.15661456  0.23296475  0.02149654 -0.07168935  0.03008167
#   -0.01506153  0.0704124   0.03533587 -0.00024379]
#  [ 0.00355438  0.07115156  0.02149654  0.20458075  0.00798919  0.18973994
#    0.08918232  0.03191677  0.07305919 -0.07019431]
#  [-0.09601458  0.07242595 -0.07168935  0.00798919  0.20534858 -0.0244321
#   -0.09154298  0.04552512  0.01293609  0.11445449]
#  [ 0.04310704  0.05466134  0.03008167  0.18973994 -0.0244321   0.20804457
#    0.1102321  -0.00291464  0.03953396 -0.1023953 ]
#  [ 0.01562474  0.04481084 -0.01506153  0.08918232 -0.09154298  0.1102321
#    0.19427639 -0.08135503 -0.02481991 -0.11032288]
#  [-0.02591271 -0.0430408   0.0704124   0.03191677  0.04552512 -0.00291464
#   -0.08135503  0.20784861  0.15843329  0.06904436]
#  [-0.04562492  0.00233283  0.03533587  0.07305919  0.01293609  0.03953396
#   -0.02481991  0.15843329  0.20318013  0.00192579]
#  [-0.06052731 -0.02437924 -0.00024379 -0.07019431  0.11445449 -0.1023953
#   -0.11032288  0.06904436  0.00192579  0.20982149]]

In [26]:
autograd_NTK

array([[ 1.2134769 , -0.52676934,  0.7556085 , -0.04334489, -0.4261384 ,
         0.12994415,  0.05166331, -0.09718898, -0.19393545, -0.2207354 ],
       [-0.52676934,  1.170739  , -0.6912442 ,  0.3135066 ,  0.30509868,
         0.2326821 ,  0.16158278, -0.17250077,  0.01924271, -0.06059065],
       [ 0.7556085 , -0.6912442 ,  1.360573  ,  0.05424047, -0.35490948,
         0.11159445, -0.03115758,  0.2816458 ,  0.13358329, -0.04844324],
       [-0.04334489,  0.3135066 ,  0.05424047,  0.85371536,  0.0809916 ,
         0.74862355,  0.31674808,  0.1227646 ,  0.27802402, -0.20531508],
       [-0.4261384 ,  0.30509868, -0.35490948,  0.0809916 ,  1.1267519 ,
        -0.05709318, -0.35211247,  0.14757422,  0.00431177,  0.48566008],
       [ 0.12994415,  0.2326821 ,  0.11159445,  0.74862355, -0.05709318,
         0.8303773 ,  0.40138292, -0.0151794 ,  0.13875796, -0.32825872],
       [ 0.05166331,  0.16158278, -0.03115758,  0.31674808, -0.35211247,
         0.40138292,  0.8189769 , -0.32440397

In [27]:
#!!!! SUCCESS!
np.allclose(NTK_easy,autograd_NTK,atol=0.0000001)

True